In [3]:
# 安装必要库（注意 PyWavelets 的正确包名）
!pip install --upgrade pip
!pip install mne PyWavelets tensorflow wget matplotlib scipy

import os
import numpy as np
import matplotlib.pyplot as plt
import mne
import pywt  # 现在可以正确导入 PyWavelets
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

files_to_download = [
    "chb08_01.edf",
    "chb08_01.seizures",
    # 可添加其他文件，例如：
    # "chb08_02.edf",
    # "chb08_02.seizures"
]

# 使用 wget 下载文件（需替换为你的 PhysioNet 账号密码）
username = "shushupi49"  # 替换为你的账号
password = "Ysppiaoliuping49"  # 替换为你的密码

base_url = "https://physionet.org/files/chbmit/1.0.0/chb08/"
for file in files_to_download:
    url = f"{base_url}{file}"
    !wget --user {username} --password {password} {url}

--2025-05-14 13:59:11--  https://physionet.org/files/chbmit/1.0.0/chb08/chb08_01.edf
Resolving physionet.org (physionet.org)... 18.18.42.54
Connecting to physionet.org (physionet.org)|18.18.42.54|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-05-14 13:59:11 ERROR 404: Not Found.

--2025-05-14 13:59:11--  https://physionet.org/files/chbmit/1.0.0/chb08/chb08_01.seizures
Resolving physionet.org (physionet.org)... 18.18.42.54
Connecting to physionet.org (physionet.org)|18.18.42.54|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-05-14 13:59:12 ERROR 404: Not Found.



In [4]:
def parse_seizure_times(seizure_file):
    """解析 .seizures 文件获取发作时间"""
    seizure_times = []
    with open(seizure_file, 'r') as f:
        lines = f.readlines()
        for i, line in enumerate(lines):
            if 'Seizure Start' in line:
                start = int(line.split(': ')[1].strip())
                end = int(lines[i+1].split(': ')[1].strip())
                seizure_times.append((start, end))
    return seizure_times

# 示例：解析 chb08_01.seizures
seizure_times = parse_seizure_times("chb08_01.seizures")
print("发作时间段:", seizure_times)

FileNotFoundError: [Errno 2] No such file or directory: 'chb08_01.seizures'

In [ ]:
def create_cwt_image(data, fs, filename, size=(128, 128)):
    """生成小波变换图像"""
    scales = np.arange(1, 128)
    coefficients, _ = pywt.cwt(data, scales, 'morl')

    plt.figure(figsize=(4,4))
    plt.imshow(np.abs(coefficients), aspect='auto', cmap='jet')
    plt.axis('off')
    plt.savefig(filename, bbox_inches='tight', pad_inches=0)
    plt.close()

# 创建存储目录
os.makedirs("seizure_images", exist_ok=True)
os.makedirs("non_seizure_images", exist_ok=True)

# 读取 EDF 文件
raw = mne.io.read_raw_edf("chb08_01.edf", preload=True)
sfreq = raw.info['sfreq']
window_length = 10  # 10秒窗口（单位：秒）

# 生成发作片段图像
seizure_count = 0
for start, end in seizure_times:
    duration = end - start
    for t in np.arange(start, end - window_length, window_length):
        data, _ = raw[:, int(t*sfreq):int((t+window_length)*sfreq)]
        create_cwt_image(data[0], sfreq, f"seizure_images/seizure_{seizure_count}.png")
        seizure_count += 1
        if seizure_count >= 50:
            break

# 生成非发作片段图像（从文件开头随机选取）
non_seizure_count = 0
total_duration = raw.times[-1]  # 总时长（秒）
for _ in range(100):  # 确保足够采样
    t = np.random.uniform(0, total_duration - window_length)
    if not any(start <= t <= end for (start, end) in seizure_times):
        data, _ = raw[:, int(t*sfreq):int((t+window_length)*sfreq)]
        create_cwt_image(data[0], sfreq, f"non_seizure_images/non_seizure_{non_seizure_count}.png")
        non_seizure_count += 1
        if non_seizure_count >= 50:
            break

In [ ]:
# 构建模型
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 数据增强
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    '/content/',  # 图像根目录（包含 seizure_images 和 non_seizure_images）
    target_size=(128,128),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    '/content/',
    target_size=(128,128),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# 训练模型
history = model.fit(
    train_generator,
    epochs=15,
    validation_data=val_generator
)

# 保存模型
model.save('eeg_seizure_cnn.h5')